<a href="https://colab.research.google.com/github/seunghyeon2776/98/blob/master/architecture.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install dataset

In [ ]:
import os
import glob
import numpy as np
import cv2
from sklearn.utils import shuffle

def load_train(train_path, image_size, classes):
    images = []
    labels = []
    ids = []
    cls = []

    print('Reading training images')
    for fld in classes:
        index = classes.index(fld)
        print('Loading {} files (Index: {})'.format(fld, index))
        path = os.path.join(train_path, fld, '*g')
        files = glob.glob(path)
        for fl in files:
            image = cv2.imread(fl)
            image = cv2.resize(image, (image_size, image_size), cv2.INTER_LINEAR)
            images.append(image)
            label = np.zeros(len(classes))
            label[index] = 1.0
            labels.append(label)
            flbase = os.path.basename(fl)
            ids.append(flbase)
            cls.append(fld)
    images = np.array(images)
    labels = np.array(labels)
    ids = np.array(ids)
    cls = np.array(cls)
  

    return images, labels, ids, cls


def load_test(test_path, image_size):
    path = os.path.join(test_path, '*g')
    files = sorted(glob.glob(path))

    X_test = []
    X_test_id = []
    print("Reading test images")
    for fl in files:
        
        img = cv2.imread(fl)
        img = cv2.resize(img, (image_size, image_size), cv2.INTER_LINEAR)
        X_test.append(img)
        X_test_id.append(flbase)
        flbase = os.path.basename(fl)

    X_test = np.array(X_test, dtype=np.uint8)
    X_test = X_test.astype('float32')
    X_test = X_test / 255
    X_test_id = np.array(X_test_id)

    return X_test, X_test_id


class DataSet(object):

    def __init__(self, images, labels, ids, cls):
        """Construct a DataSet. one_hot arg is used only if fake_data is true."""

        self._num_examples = images.shape[0]

        # Convert shape from [num examples, rows, columns, depth]
        # to [num examples, rows*columns] (assuming depth == 1)
        # Convert from [0, 255] -> [0.0, 1.0].

        images = images.astype(np.float32)
        images = np.multiply(images, 1.0 / 255.0)

        self._images = images
        self._labels = labels
        self._ids = ids
        self._cls = cls
        self._epochs_completed = 0
        self._index_in_epoch = 0

    @property
    def images(self):
        return self._images

    @property
    def labels(self):
        return self._labels

    @property
    def ids(self):
        return self._ids

    @property
    def cls(self):
        return self._cls

    @property
    def num_examples(self):
        return self._num_examples

    @property
    def epochs_completed(self):
        return self._epochs_completed

    def next_batch(self, batch_size):
        """Return the next `batch_size` examples from this data set."""
        start = self._index_in_epoch
        self._index_in_epoch += batch_size

        if self._index_in_epoch > self._num_examples:
            # Finished epoch
            self._epochs_completed += 1

            # # Shuffle the data (maybe)
            # perm = np.arange(self._num_examples)
            # np.random.shuffle(perm)
            # self._images = self._images[perm]
            # self._labels = self._labels[perm]
            # Start next epoch

            start = 0
            self._index_in_epoch = batch_size
            assert batch_size <= self._num_examples
        end = self._index_in_epoch

        return self._images[start:end], self._labels[start:end], self._ids[start:end], self._cls[start:end]


def read_train_sets(train_path, image_size, classes, validation_size=0):
    class DataSets(object):
        pass

    data_sets = DataSets()

    images, labels, ids, cls = load_train(train_path, image_size, classes)
    images, labels, ids, cls = shuffle(images, labels, ids, cls)  # shuffle the data

    if isinstance(validation_size, float):
        validation_size = int(validation_size * images.shape[0])

    train_images = images
    train_labels = labels
    train_ids = ids
    train_cls = cls

    data_sets.train = DataSet(train_images, train_labels, train_ids, train_cls)

    return data_sets

In [ ]:
def load_validation(validation_path, image_size, classes):
    images2 = []
    labels2 = []
    ids2 = []
    cls2 = []

    print('Reading validation images')
    for fld in classes:
        index = classes.index(fld)
        print('Loading {} files (Index: {})'.format(fld, index))
        path = os.path.join(validation_path, fld, '*g')
        files = glob.glob(path)
        for fl in files:
            image = cv2.imread(fl)
            image = cv2.resize(image, (image_size, image_size), cv2.INTER_LINEAR)
            images2.append(image)
            label = np.zeros(len(classes))
            label[index] = 1.0
            labels2.append(label)
            flbase = os.path.basename(fl)
            ids2.append(flbase)
            cls2.append(fld)
    images2 = np.array(images2)
    labels2 = np.array(labels2)
    ids2 = np.array(ids2)
    cls2 = np.array(cls2)
  

    return images2, labels2, ids2, cls2


def read_validation_sets(validation_path, image_size, classes, validation_size=0):
    class DataSets(object):
        pass

    data_sets = DataSets()

    images2, labels2, ids2, cls2 = load_validation(validation_path, image_size, classes)
    images2, labels2, ids2, cls2 = shuffle(images2, labels2, ids2, cls2)  # shuffle the data

    if isinstance(validation_size, float):
        validation_size = int(validation_size * images2.shape[0])

    validation_images = images2
    validation_labels = labels2
    validation_ids = ids2
    validation_cls = cls2

    data_sets.valid = DataSet(validation_images, validation_labels, validation_ids, validation_cls)

    return data_sets

def load_test(test_path, image_size, classes):
    images3 = []
    labels3 = []
    ids3 = []
    cls3 = []

    print('Reading validation images')
    for fld in classes:
        index = classes.index(fld)
        print('Loading {} files (Index: {})'.format(fld, index))
        path = os.path.join(test_path, fld, '*g')
        files = glob.glob(path)
        for fl in files:
            image = cv2.imread(fl)
            image = cv2.resize(image, (image_size, image_size), cv2.INTER_LINEAR)
            images3.append(image)
            label = np.zeros(len(classes))
            label[index] = 1.0
            labels3.append(label)
            flbase = os.path.basename(fl)
            ids3.append(flbase)
            cls3.append(fld)
    images3 = np.array(images3)
    labels3 = np.array(labels3)
    ids3 = np.array(ids3)
    cls3 = np.array(cls3)
  

    return images3, labels3, ids3, cls3


def read_test_sets(test_path, image_size, classes, validation_size=0):
    class DataSets(object):
        pass

    data_sets = DataSets()

    images3, labels3, ids3, cls3 = load_test(test_path, image_size, classes)
    images3, labels3, ids3, cls3 = shuffle(images3, labels3, ids3, cls3)  # shuffle the data

    if isinstance(validation_size, float):
        validation_size = int(validation_size * images3.shape[0])

    test_images = images3
    test_labels = labels3
    test_ids = ids3
    test_cls = cls3

    data_sets.test = DataSet(test_images, test_labels, test_ids, test_cls)

    return data_sets

In [ ]:
import time
import math
import random

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow.compat.v1 as tf
import cv2
import dataset
import os
import keras

from sklearn.metrics import confusion_matrix
from datetime import timedelta
import seaborn as sn

%matplotlib inline

tf.disable_v2_behavior()

filter_size0 = 3
num_filters0 = 3

filter_size1 = 3
num_filters1 = 64

filter_size2 = 3
num_filters2 = 128

filter_size3 = 3
num_filters3 = 256

filter_size4 = 3
num_filters4 = 512

filter_size5 = 3
num_filters5 = 512

filter_size6 = 7
num_filters6 = 512

# Fully-connected layer.
fc_size = 4096
fc_size2 = 1000           

# Number of color channels for the images: 1 channel for gray-scale.
num_channels = 3

# image dimensions (only squares for now)
img_size = 224

# Size of image when flattened to a single dimension
img_size_flat = img_size * img_size * num_channels

# Tuple with height and width of images used to reshape arrays.
img_shape = (img_size, img_size)

# class info
classes = ['1.Cancer', '2.Precancer', '3.Inflammatory', '4.Normal']
num_classes = len(classes)

# batch size
batch_size = 32

# validation split
validation_size = 0

# how long to wait after validation loss stops improving before terminating training
early_stopping = None

In [ ]:
train_path = '/content/drive/Shareddrives/CTRC/HM_Color/train'
validation_path = '/content/drive/Shareddrives/CTRC/HM_Color/validation'
test_path = '/content/drive/Shareddrives/CTRC/HM_Color/test'
checkpoint_dir = '/content/drive/MyDrive/model2/'

data = read_train_sets(train_path, img_size, classes, validation_size=validation_size)
data2 = read_validation_sets(validation_path, img_size, classes, validation_size=validation_size)
data3 = read_test_sets(test_path, img_size, classes, validation_size=validation_size)


print("Size of:")
print("- Training-set:\t\t{}".format(len(data.train.labels)))
print("- Validation-set:\t{}".format(len(data2.valid.labels)))
print("- Test-set:\t\t{}".format(len(data3.test.labels)))

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(num_filters1, filter_size1, activation='relu',
                           kernel_regularizer=tf.keras.regularizers.l2(0.001),
                           input_shape=(img_size, img_size, num_channels)),
    tf.keras.layers.Conv2D(num_filters1, filter_size1, activation='relu',
                           kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    tf.keras.layers.Dropout(rate=0.25),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(num_filters2, filter_size2, activation='relu',
                           kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    tf.keras.layers.Conv2D(num_filters2, filter_size2, activation='relu',
                           kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    tf.keras.layers.Dropout(rate=0.25),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(num_filters3, filter_size3, activation='relu',
                           kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    tf.keras.layers.Conv2D(num_filters3, filter_size3, activation='relu',
                           kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    tf.keras.layers.Conv2D(num_filters3, filter_size3, activation='relu',
                           kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    tf.keras.layers.Dropout(rate=0.25),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(num_filters4, filter_size4, activation='relu',
                           kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    tf.keras.layers.Conv2D(num_filters4, filter_size4, activation='relu',
                           kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    tf.keras.layers.Conv2D(num_filters4, filter_size4, activation='relu',
                           kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    tf.keras.layers.Dropout(rate=0.25),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(num_filters5, filter_size5, activation='relu',
                           kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    tf.keras.layers.Conv2D(num_filters5, filter_size5, activation='relu',
                           kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    tf.keras.layers.Conv2D(num_filters5, filter_size5, activation='relu',
                           kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    tf.keras.layers.Dropout(rate=0.25),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(fc_size, activation='relu'),
    tf.keras.layers.Dense(fc_size2, activation='relu'),
    tf.keras.layers.Dense(num_classes)
])

In [ ]:
from keras.utils import plot_model
plot_model(model, to_file='model.png')